# Gerekli Kütüphaneler

In [ ]:
import numpy as np  # NumPy kütüphanesini içe aktar
import pandas as pd  # Pandas kütüphanesini içe aktar
import matplotlib.pyplot as plt  # Matplotlib kütüphanesinden pyplot'ı içe aktar
import tensorflow as tf  # TensorFlow kütüphanesini içe aktar
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Görüntü ön işleme için ImageDataGenerator'ı içe aktar
from tensorflow.keras.models import Sequential  # Sequential model mimarisini içe aktar
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D  # Gerekli katmanları içe aktar
from sklearn.model_selection import train_test_split  # Veriyi eğitim ve test setlerine ayırmak için train_test_split'i içe aktar
from sklearn.preprocessing import LabelEncoder  # Etiketleri sayısal değerlere dönüştürmek için LabelEncoder'ı içe aktar
import os  # OS (işletim sistemi) kütüphanesini içe aktar
from tensorflow.keras.callbacks import EarlyStopping  # Erken durdurma callback'ini içe aktar
from sklearn.metrics import classification_report  # Sınıflandırma raporunu oluşturmak için gerekli kütüphaneyi içe aktar
from sklearn.metrics import confusion_matrix  # Karışıklık matrisini oluşturmak için gerekli kütüphaneyi içe aktar
import seaborn as sns  # Seaborn kütüphanesini içe aktar

# Veri Yükleme ve Etiket Çıkarma

In [ ]:
# Balık veri kümesinin dizini
fish_image_dir = '/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset'  
classes = []  # Sınıfları saklamak için boş bir liste oluştur

# Veri kümesindeki dizinleri kontrol et ve sınıfları ekle
for i in os.listdir(fish_image_dir):
    if '.' not in i:  # Dosya ismi değilse (sadece klasör ise)
        classes.append(i)  # Sınıf adını listeye ekle
print(classes)  # Sınıf isimlerini ekrana yazdır

# Mevcut dataset ve model yüklemesi

In [ ]:
label = []  # Etiketleri saklamak için boş bir liste oluştur
path = []  # Görüntü yollarını saklamak için boş bir liste oluştur

# Veri dizininde döngü yaparak etiket ve dosya yollarını oluştur
for dir_name, _, filenames in os.walk(fish_image_dir):
    png_files = [f for f in filenames if f.endswith('.png')]  # .png uzantılı dosyaları seç
    for filename in png_files:
        if 'GT' not in os.path.split(dir_name)[-1]:  # 'GT' klasörünü hariç tut
            label.append(os.path.split(dir_name)[-1])  # Klasör ismini etiket olarak al
            path.append(os.path.join(dir_name, filename))  # Tam dosya yolunu al

# DataFrame oluşturma
data = pd.DataFrame({
    'path': path,  # Dosya yollarını saklayan sütun
    'label': label  # Etiketleri saklayan sütun
})

# Dataset Kontrolü

In [ ]:
data.info()  # Veri çerçevesinin bilgilerini yazdır (sütun sayısı, veri tipleri, vs.)
data['label'].value_counts()  # Her etiketin kaç kez geçtiğini say

# Veriyi eğitim ve test setlerine ayırma

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2, stratify=data['label'], random_state=42)  
# Veri setini %80 eğitim, %20 test olarak ayır

# Sınıfların Görselle Örneklenmesi

In [ ]:
# Örnek görüntüleri görselleştirme
plt.figure(figsize=(15, 12))  # Görselleştirme için bir figür oluştur
for idx, label in enumerate(data['label'].unique()):  # Her benzersiz etiket için döngü
    plt.subplot(3, 3, idx + 1)  # 3x3'lük alt grafik oluştur
    plt.imshow(plt.imread(data[data['label'] == label].iloc[0, 0]))  # İlk görüntüyü oku ve göster
    plt.title(label)  # Alt grafiğin başlığına etiketi yaz
    plt.axis('off')  # Eksenleri kapat
plt.show()  # Görselleştirmeyi göster

# Görüntüleri Yükleme

In [ ]:
def load_image(img_path):  
    img = tf.io.read_file(img_path)  # Görüntüyü dosyadan oku
    img = tf.image.decode_png(img, channels=3)  # PNG formatında decode et
    img = tf.image.resize(img, [128, 128])  # Görüntüyü 128x128 boyutuna yeniden boyutlandır
    img = img / 255.0  # Piksel değerlerini 0-1 aralığına normalize et
    return img  # İşlenmiş görüntüyü döndür

# Verileri Tensorflow Veri Setlerine Dönüştürme

In [ ]:
# Eğitim görüntülerini yükle
train_images = [load_image(path) for path in train_data['path']]  # Eğitim görüntülerini yükle
train_labels = train_data['label'].apply(classes.index).values  # Etiketleri sayısal değerlere dönüştür

# Test görüntülerini yükle
test_images = [load_image(path) for path in test_data['path']]  # Test görüntülerini yükle
test_labels = test_data['label'].apply(classes.index).values  # Test etiketlerini sayısal değerlere dönüştür

# TensorFlow veri seti oluşturma
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(32)  # Eğitim veri seti oluştur ve 32'li gruplara ayır
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(32)  # Test veri seti oluştur ve 32'li gruplara ayır

# Sinir Ağı Modeli Oluşturma

In [ ]:
# Modeli oluşturma
model = tf.keras.Sequential()  # Sequential model başlangıcı
input_layer = tf.keras.layers.Input(shape=(128, 128, 3))  # Giriş katmanı, 128x128 boyutunda 3 kanal

# İlk konvolüsyon katmanı ve max pooling katmanı
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3)))  
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))  

# İkinci konvolüsyon katmanı ve max pooling katmanı
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3)))  
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))  

# Üçüncü konvolüsyon katmanı ve max pooling katmanı
model.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3)))  
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))  

model.add(tf.keras.layers.Flatten())  # Görüntüyü düzleştir

model.add(tf.keras.layers.Dense(128, activation='relu'))  # Tam bağlantılı katman, 128 nöron

# Son katman, sınıf sayısına göre
model.add(tf.keras.layers.Dense(len(classes), activation='softmax'))  # Sınıf sayısına göre softmax ile son katmanı ekle

# Modeli derleme
model.compile(optimizer='adam',  # Adam optimizasyon algoritmasını kullan
              loss='sparse_categorical_crossentropy',  # Kayıp fonksiyonu olarak sparse categorical crossentropy kullan
              metrics=['accuracy'])  # Başarı metriği olarak doğruluğu kullan

# Hata Ayıklama ve Sinir Ağı Modelinin Eğitimi

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',  # 'val_loss' değerini izlemek için
    patience=5,  # Doğrulama kaybı iyileşmezse 5 epoch bekle
    restore_best_weights=True  # En iyi ağırlıkları geri yüklemek için
)

# Modeli eğitme
history = model.fit(
    train_ds,  # Eğitim verisi
    validation_data=test_ds,  # Doğrulama verisi
    epochs=20,  # 20 epoch boyunca eğitim
    callbacks=[early_stopping]  # Erken durdurma callback'i kullan
)

# Eğitim ve Doğrulama Grafiğinin Çizimi

In [ ]:
# Eğitim ve doğrulama sonuçlarını al
acc = history.history['accuracy']  # Eğitim doğruluğunu al
val_acc = history.history['val_accuracy']  # Doğrulama doğruluğunu al
loss = history.history['loss']  # Eğitim kaybını al
val_loss = history.history['val_loss']  # Doğrulama kaybını al

# Sonuçları görselleştirme
plt.figure(figsize=(12, 6))  # 12x6 boyutlarında bir figür oluştur
plt.subplot(1, 2, 1)  # 1 satır, 2 sütun, 1. grafik
plt.plot(acc, label='Training Accuracy')  # Eğitim doğruluğunu çiz
plt.plot(val_acc, label='Validation Accuracy')  # Doğrulama doğruluğunu çiz
plt.legend()  # Legend ekle
plt.title('Training and Validation Accuracy')  # Başlık ekle

plt.subplot(1, 2, 2)  # 1 satır, 2 sütun, 2. grafik
plt.plot(loss, label='Training Loss')  # Eğitim kaybını çiz
plt.plot(val_loss, label='Validation Loss')  # Doğrulama kaybını çiz
plt.legend()  # Legend ekle
plt.title('Training and Validation Loss')  # Başlık ekle
plt.show()  # Görselleştirmeyi göster

# Modelin Test Verileri Üzerinde Değerlendirilmesi

In [ ]:
# Test setini değerlendirme
test_loss, test_accuracy = model.evaluate(test_ds, verbose=0)  # Test verisi üzerinde kaybı ve doğruluğu değerlendir
print('Test Loss:', test_loss)  # Test kaybını yazdır
print('Test Accuracy:', test_accuracy)  # Test doğruluğunu yazdır

In [ ]:
# Test verisi üzerinde tahminler yapma
y_pre_prob = model.predict(test_ds)  # Test veri seti üzerinde tahmin yap
y_pre_class = np.argmax(y_pre_prob, axis=1)  # En yüksek olasılığa sahip sınıfı seç

# Doğru etiketler
y_true = test_labels  # Test setindeki gerçek etiketler

# Sınıflandırma raporu
print(classification_report(y_true, y_pre_class, target_names=classes))  # Sınıflandırma raporunu yazdır


In [ ]:
# Karışıklık matrisini hesapla
cm = confusion_matrix(y_true, y_pre_class)  # Gerçek ve tahmin edilen etiketleri kullanarak karışıklık matrisini oluştur

# Karışıklık matrisini görselleştirme
plt.figure(figsize=(10, 8))  # Görselleştirme için bir figür oluştur
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',  # Seaborn ile ısı haritası oluştur
             xticklabels=classes, yticklabels=classes)  # Sınıf isimlerini eksenlerde göster
plt.title('Confusion Matrix')  # Başlık ekle
plt.xlabel('Predicted')  # X eksenine etiket ekle
plt.ylabel('True')  # Y eksenine etiket ekle
plt.show()  # Görselleştirmeyi göster

# Veri Setindeki Görüntüyü Tahmin Etme

In [ ]:
from PIL import Image 
# Test edilecek resmin yolu
image_path = '/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset/Hourse Mackerel/Hourse Mackerel/00030.png'

# Görüntüyü aç, boyutlandır ve normalize et
test_img = Image.open(image_path)  # Görüntüyü aç
test_img = test_img.resize((128, 128))  # Modelin beklediği boyut (128x128) olacak şekilde yeniden boyutlandır
test_img_array = np.array(test_img).astype('float32') / 255  # NumPy dizisine dönüştür ve normalize et

# Modelin beklediği formatta giriş oluşturmak için boyutları genişlet
test_img_array = np.expand_dims(test_img_array, axis=0)  # (1, 128, 128, 3) boyutuna genişlet

# Modelden tahmin yapma
predictions = model.predict(test_img_array)  # Modelden tahmin al
# Tahmin edilen sınıf indeksini bulma
predicted_class_index = np.argmax(predictions)  # En yüksek olasılığa sahip sınıfın indeksini bul

# Dataset içindeki etiket isimlerini alıyoruz
class_names = classes  # Sınıf isimlerini elde ediyoruz

# Tahmin edilen sınıf ismi
predicted_class = class_names[predicted_class_index]  # Tahmin edilen sınıf ismini bul

# Tahmin sonucunu görselleştirme (Pillow ile)
plt.imshow(test_img)  # test_img PIL imaj nesnesidir, NumPy dizisi değil
plt.title(f'Predicted Class: {predicted_class}')  # Tahmin edilen sınıf ismini başlık olarak ayarla
plt.axis('off')  # Eksenleri gizle
plt.show()  # Görselleştirmeyi göster